In [1]:
import argparse, csv
from tensorflow import keras
from tensorflow.python.keras.models import model_from_json
from keras.utils import to_categorical
import pandas as pd

from tools.metadata_tools import *

Using TensorFlow backend.


In [2]:
val = '../Results/json_detection'
train = '../Results/json_detection/train'
threshold = 0.001

matrix_metadata = metadata_to_matrix(val, "json")
val_data = matrix_metadata[:,:14].astype(np.float32)
val_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))
val_data[val_data < threshold] = 0
val_data[val_data > threshold] = 1
val_data = val_data.astype(np.uint8)

matrix_metadata = metadata_to_matrix(train, "json")
train_data = matrix_metadata[:,:14].astype(np.float32)
train_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))
train_data[train_data < threshold] = 0
train_data[train_data >= threshold] = 1
train_data = train_data.astype(np.uint8)

from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(train_data, train_label, train_size = 0.8)

In [3]:
from monumai.monument import Monument
arch_el_names = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]
style_names = Monument.STYLES_HOTONE_ENCODE

In [4]:
objectProperties = ["isTypicalOfStyle", "hasArchElement", "exhibitsArchStyle"]

import csv
with open('train.txt', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    for i in range(len(arch_el_names)):
        for style, el in Monument.TRUE_ELEMENT_DIC.items():
            if style == 'baroque':
                style = 'B'
            if style == 'hispanic-muslim':
                style = 'M'
            if style == 'renaissance':
                style = 'R'
            if style == 'gothic':
                style = 'G'
            if arch_el_names[i] in el:
                writer.writerow([arch_el_names[i], objectProperties[0], style])
                continue
    for i in range(len(train_data)):
        line = train_data[i]
        for k in range(len(line)):
            if line[k]:
                writer.writerow([i, objectProperties[1], arch_el_names[k]])
    for i in range(len(val_data)):
        line = val_data[i]
        for k in range(len(line)):
            if line[k]:
                writer.writerow([i + len(train_data), objectProperties[1], arch_el_names[k]])  
    for i in range(len(test_data)):
        line = test_data[i]
        for k in range(len(line)):
            if line[k]:
                writer.writerow([i + len(train_data) + len(val_data), objectProperties[1], arch_el_names[k]])            
    for i in range(len(train_data)):
        k = np.argmax(train_label[i])
        writer.writerow([i , objectProperties[2], style_names[k]])


In [5]:
with open('val.txt', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')       
    for i in range(len(val_data)):
        k = np.argmax(val_label[i])
        writer.writerow([i + len(train_data), objectProperties[2], style_names[k]])
with open('test.txt', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')       
    for i in range(len(test_data)):
        k = np.argmax(test_label[i])
        writer.writerow([i + len(train_data) + len(val_data) , objectProperties[2], style_names[k]])

In [6]:
val = '../Results/json_detection'
train = '../Results/json_detection/train'

matrix_metadata = metadata_to_matrix(val, "json")
val_data = matrix_metadata[:,:14].astype(np.float32)
val_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))

matrix_metadata = metadata_to_matrix(train, "json")
train_data = matrix_metadata[:,:14].astype(np.float32)
train_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))

In [8]:
import csv
import numpy as np
with open('sam_tab_train.txt', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    header = arch_el_names + style_names
    writer.writerow(header)
    for i in range(len(train_data)):
        writer.writerow(np.concatenate((train_data[i],train_label[i])))